## Data Preperation

In [52]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
from sklearn.preprocessing import MinMaxScaler

pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("display.float_format",lambda x: "%3f"%x)

In [53]:
df_ = pd.read_csv(r"C:\Users\melih.oznalbant\OneDrive - Litum Technologies\Desktop\pythonProject1\Superstore.csv", encoding='windows-1254')
df = df_.copy()

In [54]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.960000,2,0.000000,41.913600
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.940000,3,0.000000,219.582000
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.620000,2,0.000000,6.871400
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.577500,5,0.450000,-383.031000
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.368000,2,0.200000,2.516400


In [55]:
df.isnull().sum()

Row ID           0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64

In [56]:
df.shape

(9994, 21)

In [57]:
df.nunique()

Row ID           9994
Order ID         5009
Order Date       1237
Ship Date        1334
Ship Mode           4
Customer ID       793
Customer Name     793
Segment             3
Country             1
City              531
State              49
Postal Code       631
Region              4
Product ID       1862
Category            3
Sub-Category       17
Product Name     1850
Sales            5825
Quantity           14
Discount           12
Profit           7287
dtype: int64

In [58]:
df.dtypes

Row ID             int64
Order ID          object
Order Date        object
Ship Date         object
Ship Mode         object
Customer ID       object
Customer Name     object
Segment           object
Country           object
City              object
State             object
Postal Code        int64
Region            object
Product ID        object
Category          object
Sub-Category      object
Product Name      object
Sales            float64
Quantity           int64
Discount         float64
Profit           float64
dtype: object

In [59]:
for i in df.columns:
    if "Date" in i:
        df[i] = pd.to_datetime(df[i])


In [60]:
df.dtypes

Row ID                    int64
Order ID                 object
Order Date       datetime64[ns]
Ship Date        datetime64[ns]
Ship Mode                object
Customer ID              object
Customer Name            object
Segment                  object
Country                  object
City                     object
State                    object
Postal Code               int64
Region                   object
Product ID               object
Category                 object
Sub-Category             object
Product Name             object
Sales                   float64
Quantity                  int64
Discount                float64
Profit                  float64
dtype: object

In [61]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Row ID,9994.000000,4997.500000,2885.163629,1.000000,2499.250000,4997.500000,7495.750000,9994.000000
Postal Code,9994.000000,55190.379428,32063.693350,1040.000000,23223.000000,56430.500000,90008.000000,99301.000000
Sales,9994.000000,229.858001,623.245101,0.444000,17.280000,54.490000,209.940000,22638.480000
Quantity,9994.000000,3.789574,2.225110,1.000000,2.000000,3.000000,5.000000,14.000000
Discount,9994.000000,0.156203,0.206452,0.000000,0.000000,0.200000,0.200000,0.800000
Profit,9994.000000,28.656896,234.260108,-6599.978000,1.728750,8.666500,29.364000,8399.976000


In [62]:
today_date = df["Order Date"].max() + dt.timedelta(days=2)

In [63]:
rfm = df.groupby("Customer ID").agg({"Order Date" : lambda date: (today_date - date.max()).days, #days burada sonucu gün cinsinden ifade et anlamındadır
                                     "Order ID" : lambda num: num.nunique(),
                                     "Sales" : lambda TotalPrice: TotalPrice.sum()})

In [64]:
rfm.columns = ["recency","frequency","monetary"]

In [65]:
rfm.head()

,recency,frequency,monetary
Customer ID,,,
AA-10315,186,5,5563.560000
AA-10375,21,9,1056.390000
AA-10480,261,4,1790.512000
AA-10645,57,6,5086.935000
AB-10015,417,3,886.156000


In [66]:
rfm.shape

(793, 3)

In [67]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,793.000000,148.802018,186.211051,2.000000,32.000000,77.000000,185.000000,1167.000000
frequency,793.000000,6.316520,2.550885,1.000000,5.000000,6.000000,8.000000,17.000000
monetary,793.000000,2896.848500,2628.670117,4.833000,1146.050000,2256.394000,3785.276000,25043.050000


In [68]:
rfm["recency_score"] = pd.qcut(rfm["recency"],5,labels=[5,4,3,2,1])

In [69]:
rfm["frequency_score"] = pd.qcut(rfm["frequency"],5,labels=[1,2,3,4,5])

In [70]:
rfm["monetary_score"] = pd.qcut(rfm["monetary"],5,labels=[1,2,3,4,5])

In [71]:
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score
Customer ID,,,,,,
AA-10315,186,5,5563.560000,2,2,5
AA-10375,21,9,1056.390000,5,5,2
AA-10480,261,4,1790.512000,1,1,3
AA-10645,57,6,5086.935000,3,3,5
AB-10015,417,3,886.156000,1,1,1


In [72]:
rfm["RFM_Score"] = rfm.apply(lambda i: str(int(i.recency_score)) + str(int(i.frequency_score)), axis = 1)



In [73]:
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_Score
Customer ID,,,,,,,
AA-10315,186,5,5563.560000,2,2,5,22
AA-10375,21,9,1056.390000,5,5,2,55
AA-10480,261,4,1790.512000,1,1,3,11
AA-10645,57,6,5086.935000,3,3,5,33
AB-10015,417,3,886.156000,1,1,1,11


In [74]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [75]:
rfm["segment"] = rfm["RFM_Score"].replace(seg_map,regex=True)

In [76]:
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_Score,segment
Customer ID,,,,,,,,
AA-10315,186,5,5563.560000,2,2,5,22,hibernating
AA-10375,21,9,1056.390000,5,5,2,55,champions
AA-10480,261,4,1790.512000,1,1,3,11,hibernating
AA-10645,57,6,5086.935000,3,3,5,33,need_attention
AB-10015,417,3,886.156000,1,1,1,11,hibernating


In [77]:
# Customer Lifetime Value

In [78]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.960000,2,0.000000,41.913600
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.940000,3,0.000000,219.582000
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.620000,2,0.000000,6.871400
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.577500,5,0.450000,-383.031000
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.368000,2,0.200000,2.516400


In [79]:
df.columns = [i.replace(" ","_") for i in df.columns]

In [80]:
df.head()

,Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,Country,City,State,Postal_Code,Region,Product_ID,Category,Sub-Category,Product_Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.960000,2,0.000000,41.913600
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.940000,3,0.000000,219.582000
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.620000,2,0.000000,6.871400
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.577500,5,0.450000,-383.031000
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.368000,2,0.200000,2.516400


In [81]:
df_CLTV = df.groupby('Customer_ID').agg(
    {'Order_Date': [lambda Order_Date: (Order_Date.max() - Order_Date.min()).days,
                     lambda Order_Date: (today_date - Order_Date.min()).days],
     'Order_ID': lambda Order_ID: Order_ID.nunique(),
     'Sales': lambda Sales: Sales.sum()})                                      

In [82]:
df_CLTV.columns=['recency', 'T', 'frequency', 'monetary']

In [83]:
df_CLTV["monetary"]=df_CLTV["monetary"]/df_CLTV["frequency"]

In [84]:
df_CLTV["recency"]=df_CLTV["recency"]/7

In [85]:
df_CLTV["T"]=df_CLTV["T"]/7

In [86]:
df_CLTV = df_CLTV[df_CLTV["frequency"]>1]

In [87]:
df_CLTV.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,781.000000,145.001463,40.879706,0.285714,120.857143,156.428571,174.714286,205.714286
T,781.000000,165.732943,33.682675,8.142857,157.428571,172.857143,190.142857,208.428571
frequency,781.000000,6.398207,2.483060,2.000000,5.000000,6.000000,8.000000,17.000000
monetary,781.000000,460.601169,433.191229,2.416500,216.197900,362.947286,547.473429,5008.610000


In [88]:
df_CLTV.head()

,recency,T,frequency,monetary
Customer_ID,,,,
AA-10315,169.428571,196.000000,5,1112.712000
AA-10375,190.000000,193.000000,9,117.376667
AA-10480,153.857143,191.142857,4,447.628000
AA-10645,176.000000,184.142857,6,847.822500
AB-10015,142.285714,201.857143,3,295.385333


## Establishment of BG-NBD Model

### Expected Number Of Transaction

In [89]:
# Fitting the model

bgf = BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(df_CLTV["frequency"],df_CLTV["recency"],df_CLTV["T"])

<lifetimes.BetaGeoFitter: fitted with 781 subjects, a: 0.00, alpha: 192.17, b: 0.00, r: 7.50>

In [101]:
df_CLTV["exp_sales_3_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(3*4,         # haftalık data olduğu için 1 ay 4 hafta üzerinden hesaplanmıştır
                                                        df_CLTV['frequency'],
                                                        df_CLTV["recency"],
                                                        df_CLTV["T"]).sort_values(ascending=False)

In [91]:
df_CLTV["exp_sales_6_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(6*4,         # haftalık data olduğu için 1 ay 4 hafta üzerinden hesaplanmıştır
                                                        df_CLTV['frequency'],
                                                        df_CLTV["recency"],
                                                        df_CLTV["T"]).sort_values(ascending=False)

In [92]:
df_CLTV.sort_values(by = "exp_sales_3_month",ascending=False).head()

,recency,T,frequency,monetary,exp_sales_3_month,exp_sales_6_month
Customer_ID,,,,,,
EP-13915,180.142857,182.142857,17,322.238871,0.785306,1.570612
MH-18115,69.857143,70.857143,9,611.454778,0.752584,1.505168
SJ-20125,105.000000,118.857143,12,329.138000,0.752185,1.504371
BD-11320,130.142857,135.142857,12,476.543417,0.714759,1.429519
CV-12805,85.000000,86.000000,9,263.368178,0.711615,1.423230


In [93]:
df_CLTV.sort_values(by = "exp_sales_6_month",ascending=False).head()

,recency,T,frequency,monetary,exp_sales_3_month,exp_sales_6_month
Customer_ID,,,,,,
EP-13915,180.142857,182.142857,17,322.238871,0.785306,1.570612
MH-18115,69.857143,70.857143,9,611.454778,0.752584,1.505168
SJ-20125,105.000000,118.857143,12,329.138000,0.752185,1.504371
BD-11320,130.142857,135.142857,12,476.543417,0.714759,1.429519
CV-12805,85.000000,86.000000,9,263.368178,0.711615,1.423230


In [94]:
df_CLTV.head()

ggf = GammaGammaFitter(penalizer_coef=0.01)

## Establishing the Gamma-Gamma Model

### Expected Average Profit


In [95]:
# Fitting the model

ggf.fit(df_CLTV['frequency'], df_CLTV['monetary'])

df_CLTV["exp_average_value"] = ggf.conditional_expected_average_profit(df_CLTV['frequency'],
                                                                             df_CLTV['monetary'])

<lifetimes.GammaGammaFitter: fitted with 781 subjects, p: 3.64, q: 0.32, v: 3.60>

## Calculation of CLTV with BG-NBD and GG Model

In [97]:
CLTV = ggf.customer_lifetime_value(bgf,
                                       df_CLTV['frequency'],
                                       df_CLTV['recency'],
                                       df_CLTV['T'],
                                       df_CLTV['monetary'],
                                       time=6,  # 6 month
                                       freq="W",  # T's frequency knowledge (Week)
                                       discount_rate=0.01)

In [98]:
CLTV.reset_index()

,Customer_ID,clv
0,AA-10315,937.652928
1,AA-10375,129.711902
2,AA-10480,355.346175
3,AA-10645,790.850625
4,AB-10015,212.165801
5,AB-10060,1243.658283
6,AB-10105,1854.659376
7,AB-10150,161.876091
8,AB-10165,157.068846
9,AB-10255,116.321801


In [99]:
CLTV_Final = df_CLTV.merge(CLTV, on="Customer_ID", how="left")

In [100]:

CLTV_Final.sort_values(by="clv",ascending=False).head(20)

,recency,T,frequency,monetary,exp_sales_3_month,exp_sales_6_month,exp_average_value,clv
Customer_ID,,,,,,,,
SM-20320,186.285714,197.857143,5,5008.610000,0.384457,0.768914,5204.031078,4198.413602
TC-20980,107.142857,164.428571,5,3810.443600,0.420497,0.840995,3959.294681,3493.645217
RB-19360,77.428571,91.428571,6,2519.556500,0.571051,1.142103,2601.258275,3117.140014
TA-21385,162.285714,172.428571,4,3648.905000,0.378357,0.756715,3828.869255,3039.978828
CC-12370,77.571429,84.000000,5,2425.814400,0.542960,1.085920,2520.848100,2872.183050
BM-11140,128.857143,173.000000,4,2947.407500,0.377765,0.755531,3092.955322,2451.847943
GT-14635,178.142857,186.428571,3,3117.070667,0.332670,0.665340,3325.643938,2321.598981
CM-12385,140.428571,146.000000,4,2238.505000,0.407927,0.815854,2349.273089,2011.007868
CJ-12010,67.285714,94.571429,8,1395.621750,0.648492,1.296985,1429.498274,1945.298351
